In [1]:
# P2047

In [33]:
import json
import pathlib
import pydash
import requests
import time
from wikibaseintegrator import WikibaseIntegrator, wbi_login, datatypes
from wikibaseintegrator.models import Claims, Qualifiers, References, Reference
from wikibaseintegrator.wbi_config import config

def pull_duration(link):

    ''' Pull duration data from OFLC. '''

    time.sleep(4)
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:82.0) Gecko/20100101 Firefox/82.0'} 
    r = requests.get(link, headers=headers).text
    duration = r.split('<div class="field__label">Duration</div>')[1].split('</div>')[0]
    duration = duration.replace('<div class="field__item">','').replace('minutes','').strip()
    return duration

def write_duration(item, dur, ref, log):

    ''' Write duration data to Wikidata. '''

    references = References()
    reference1 = Reference()
    reference1.add(datatypes.URL(prop_nr='P854', value=ref))
    references.add(reference1)

    wbi = WikibaseIntegrator(login=login_wikidata)
    wd_item = wbi.item.get(item, mediawiki_api_url='https://www.wikidata.org/w/api.php', login=log)
    claim = datatypes.Quantity(prop_nr='P2047', amount=dur, unit='Q7727', references=references)    
    wd_item.claims.add(claim)
    wd_item.write()


with open(pathlib.Path.cwd().parents[0] / 'directory.json') as direct:
    direct = json.load(direct)

with open(pathlib.Path.home() / 'wikidata_login.json') as wd_cred:
    wd_cred = json.load(wd_cred)

login_wikidata = wbi_login.Login(user=wd_cred['username'], password=wd_cred['password'], mediawiki_api_url='https://www.wikidata.org/w/api.php')

for x in direct.keys():
    if pydash.get(direct, f'{x}.oflc'):
        oflc_link = 'https://www.classification.gov.au/'+pydash.get(direct, f'{x}.oflc')
        duration = pull_duration(oflc_link)
        print(x, 'P2047', duration)
        write_duration(x, duration, oflc_link, login_wikidata)

print('all done.')

MediaWiki login warnings messages:
* main: Subscribe to the mediawiki-api-announce mailing list at <https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/> for notice of API deprecations and breaking changes. Use [[Special:ApiFeatureUsage]] to see usage of deprecated features by your application.
* login: Main-account login via "action=login" is deprecated and may stop working without warning. To continue login with "action=login", see [[Special:BotPasswords]]. To safely continue using main-account login, see "action=clientlogin".


Q1421747 P2047 118
Q12126942 P2047 97
Q3415693 P2047 107
all done.
